In [1]:
# SKRYPT EKSPERYMENTALNY
# Uruchamia analizę algorytmu Symulowanego Wyżarzania (SA)
# dla różnych zestawów parametrów. Dla każdej konfiguracji
# wykonywanych jest kilka powtórzeń, zapisywany jest koszt,
# czas wykonania oraz średnie statystyki. Wyniki trafiają
# do pliku CSV przy użyciu save_experiment_results().

%load_ext autoreload
%autoreload 2

import time
import pandas as pd
import numpy as np
import itertools
from multiprocessing import Pool, cpu_count

from src.utils.tsp_loader import load_tsp_matrix
from src.algorithms.sa_numba import solve_tsp
from src.utils.result_saver import save_experiment_results
from src.utils.run_single_repeat import run_single_repeat


# USTAWIENIA
TSP_FILES = ["Dane_TSP_48.xlsx", "Dane_TSP_76.xlsx", "Dane_TSP_127.xlsx"]

PARAM_GRID = {
    # "T0": [4_000, 50_000, 200_000, 1_000_000],
    # "alpha": [0.98, 0.99, 0.999, 0.9999],
    "T0": [500, 1_000, 2_000, 4_000],
    "alpha": [0.98, 0.99, 0.999, 0.9999, 0.99999],
    "T_min": [0.01, 0.0001, 0.00001, 0.0000001],
    "max_iter": [25_000, 300_000, 1_000_000, 10_000_000],
    "neighborhood_type": ["swap", "two_opt", "insert"],
}

REPEATS = 5
results = []


# ROZGRZANIE NUMBA (KOMPILACJA JIT)
print("Rozgrzewanie Numba (kompilacja JIT)...")
D = load_tsp_matrix(TSP_FILES[0])
_ = solve_tsp(D, {
    "T0": 1000,
    "alpha": 0.999,
    "T_min": 0.001,
    "max_iter": 100,
    "neighborhood_type": "swap",
    "use_delta": False
})
print("Kompilacja zakończona.\n")


# GŁÓWNA PĘTLA
# Obliczamy łączną liczbę kombinacji
all_combos = list(itertools.product(
    PARAM_GRID["T0"],
    PARAM_GRID["alpha"],
    PARAM_GRID["T_min"],
    PARAM_GRID["max_iter"],
    PARAM_GRID["neighborhood_type"],
))
total = len(all_combos) * len(TSP_FILES)
counter = 0
start_total = time.perf_counter()

for tsp_file in TSP_FILES:
    print(f"\nInstancja: {tsp_file}")
    D = load_tsp_matrix(tsp_file)

    for T0 in PARAM_GRID["T0"]:
        for alpha in PARAM_GRID["alpha"]:
            for T_min in PARAM_GRID["T_min"]:
                for max_iter in PARAM_GRID["max_iter"]:
                    for neighborhood_type in PARAM_GRID["neighborhood_type"]:
                        counter += 1

                        print(
                            f"[{counter}/{total}] "
                            f"T0={T0}, "
                            f"alpha={alpha}, "
                            f"T_min={T_min}, "
                            f"max_iter={max_iter}, "
                            f"neighborhood_type={neighborhood_type}"
                        )

                        params = {
                            "T0": T0,
                            "alpha": alpha,
                            "T_min": T_min,
                            "max_iter": max_iter,
                            "neighborhood_type": neighborhood_type,
                            "use_delta": True,
                        }

                        # multiprocessing — równoległe powtórzenia
                        with Pool(processes=cpu_count()) as pool:
                            parallel_jobs = [
                                (solve_tsp, D, params) for _ in range(REPEATS)
                            ]
                            results_parallel = pool.map(run_single_repeat, parallel_jobs)

                        # ZBIERANIE
                        costs = [c for c, _, _ in results_parallel]
                        routes = [r for _, r, _ in results_parallel]
                        runtimes = [t for _, _, t in results_parallel]

                        # najlepszy koszt
                        min_cost = min(costs)

                        # pełna trasa najlepszego wyniku
                        best_route_overall = routes[costs.index(min_cost)]
                        route_str = "-".join(map(str, best_route_overall))

                        # zapis wyników
                        results.append({
                            "instance": tsp_file,
                            "T0": T0,
                            "alpha": alpha,
                            "T_min": T_min,
                            "max_iter": max_iter,
                            "neighborhood_type": neighborhood_type,
                            "mean_cost": round(np.mean(costs), 3),
                            "min_cost": round(min_cost, 3),
                            "mean_runtime": np.mean(runtimes),
                            "min_route": route_str,
                        })

# ZAPIS WYNIKÓW
end_total = time.perf_counter()
elapsed = end_total - start_total

print(f"\nŁączny czas eksperymentów: {elapsed/60:.2f} min ({elapsed:.2f} sek)")

df = pd.DataFrame(results)
save_experiment_results(df, time_seconds=int(elapsed), subfolder="SA")

print("\nNajlepsze parametry dla każdej instancji:")

instances = df["instance"].unique()

for inst in instances:
    sub = df[df["instance"] == inst]
    best_row = sub.loc[sub["mean_cost"].idxmin()]
    print(f"\n{inst}")
    # print(best_row["min_cost"], best_row.to_dict())
    print(f"odległość {best_row["min_cost"]} = {best_row.to_dict()}")

Rozgrzewanie Numba (kompilacja JIT)...
Kompilacja zakończona.


Instancja: Dane_TSP_48.xlsx
[1/2880] T0=500, alpha=0.98, T_min=0.01, max_iter=25000, neighborhood_type=swap
[2/2880] T0=500, alpha=0.98, T_min=0.01, max_iter=25000, neighborhood_type=two_opt
[3/2880] T0=500, alpha=0.98, T_min=0.01, max_iter=25000, neighborhood_type=insert
[4/2880] T0=500, alpha=0.98, T_min=0.01, max_iter=300000, neighborhood_type=swap
[5/2880] T0=500, alpha=0.98, T_min=0.01, max_iter=300000, neighborhood_type=two_opt
[6/2880] T0=500, alpha=0.98, T_min=0.01, max_iter=300000, neighborhood_type=insert
[7/2880] T0=500, alpha=0.98, T_min=0.01, max_iter=1000000, neighborhood_type=swap
[8/2880] T0=500, alpha=0.98, T_min=0.01, max_iter=1000000, neighborhood_type=two_opt
[9/2880] T0=500, alpha=0.98, T_min=0.01, max_iter=1000000, neighborhood_type=insert
[10/2880] T0=500, alpha=0.98, T_min=0.01, max_iter=10000000, neighborhood_type=swap
[11/2880] T0=500, alpha=0.98, T_min=0.01, max_iter=10000000, neighborhood_type=t